<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/creditriskplus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math # For fabs, though assertAlmostEqual is preferred

def double_vector_from_list(lst):
    dv = ql.DoubleVector(len(lst))
    for i, val in enumerate(lst):
        dv[i] = val
    return dv

def size_vector_from_list(lst):
    sv = ql.SizeVector(len(lst))
    for i, val in enumerate(lst):
        sv[i] = val
    return sv

def matrix_from_list_of_lists(lol):
    if not lol:
        return ql.Matrix(0,0)
    rows = len(lol)
    cols = len(lol[0])
    m = ql.Matrix(rows, cols)
    for i in range(rows):
        for j in range(cols):
            m[i,j] = lol[i][j] # Use m[i,j] for ql.Matrix
    return m


class CreditRiskPlusTests(unittest.TestCase):

    def test_reference_values(self):
        print("Testing extended credit risk plus model against reference values...")

        tol = 1E-8

        # Sector 1 data
        sector1_exposure_list = [1.0] * 1000
        sector1_pd_list = [0.04] * 1000
        sector1_sector_list = [0] * 1000

        # Sector 2 data
        sector2_exposure_list = [2.0] * 1000
        sector2_pd_list = [0.02] * 1000
        sector2_sector_list = [1] * 1000

        # Combine data
        exposure_list = sector1_exposure_list + sector2_exposure_list
        pd_list = sector1_pd_list + sector2_pd_list
        sector_list = sector1_sector_list + sector2_sector_list

        exposure_qv = double_vector_from_list(exposure_list)
        pd_qv = double_vector_from_list(pd_list)
        sector_qv = size_vector_from_list(sector_list)

        relative_default_variance_list = [0.75 * 0.75, 0.75 * 0.75]
        relative_default_variance_qv = double_vector_from_list(relative_default_variance_list)

        rho_data = [
            [1.0, 0.50],
            [0.50, 1.0]
        ]
        rho_qm = matrix_from_list_of_lists(rho_data)

        unit = 0.1

        cr_model = ql.CreditRiskPlus(exposure_qv, pd_qv, sector_qv,
                                     relative_default_variance_qv, rho_qm, unit)

        # Sector 1 checks
        self.assertAlmostEqual(cr_model.sectorExposures()[0], 1000.0, delta=tol,
                               msg=f"Sector 1 exposure: got {cr_model.sectorExposures()[0]}, expected 1000.0")
        self.assertAlmostEqual(cr_model.sectorExpectedLoss()[0], 40.0, delta=tol,
                               msg=f"Sector 1 expected loss: got {cr_model.sectorExpectedLoss()[0]}, expected 40.0")
        # For unexpected loss, C++ uses a larger tolerance (0.05)
        self.assertAlmostEqual(cr_model.sectorUnexpectedLoss()[0], 30.7, delta=0.05,
                               msg=f"Sector 1 unexpected loss: got {cr_model.sectorUnexpectedLoss()[0]}, expected 30.7")

        # Sector 2 checks
        self.assertAlmostEqual(cr_model.sectorExposures()[1], 2000.0, delta=tol,
                               msg=f"Sector 2 exposure: got {cr_model.sectorExposures()[1]}, expected 2000.0")
        self.assertAlmostEqual(cr_model.sectorExpectedLoss()[1], 40.0, delta=tol,
                               msg=f"Sector 2 expected loss: got {cr_model.sectorExpectedLoss()[1]}, expected 40.0")
        self.assertAlmostEqual(cr_model.sectorUnexpectedLoss()[1], 31.3, delta=0.05,
                               msg=f"Sector 2 unexpected loss: got {cr_model.sectorUnexpectedLoss()[1]}, expected 31.3")

        # Overall checks
        self.assertAlmostEqual(cr_model.exposure(), 3000.0, delta=tol,
                               msg=f"Overall exposure: got {cr_model.exposure()}, expected 3000.0")
        self.assertAlmostEqual(cr_model.expectedLoss(), 80.0, delta=tol,
                               msg=f"Overall expected loss: got {cr_model.expectedLoss()}, expected 80.0")
        # For overall unexpected loss, C++ uses tolerance 0.01
        self.assertAlmostEqual(cr_model.unexpectedLoss(), 53.1, delta=0.01,
                               msg=f"Overall unexpected loss: got {cr_model.unexpectedLoss()}, expected 53.1")

        # For overall relative default variance, C++ uses tolerance 0.001
        self.assertAlmostEqual(cr_model.relativeDefaultVariance(), 0.65 * 0.65, delta=0.001,
                               msg=f"Overall relative default variance: got {cr_model.relativeDefaultVariance()}, expected {0.65*0.65}")

        # For loss quantile, C++ uses tolerance 0.5
        self.assertAlmostEqual(cr_model.lossQuantile(0.99), 250.0, delta=0.5,
                               msg=f"Overall 99 percentile: got {cr_model.lossQuantile(0.99)}, expected 250.0")


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    # TopLevelFixture actions are generally not critical for math/model tests
    # unless they set specific global settings like evaluation date,
    # which is not the case here.
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

double_vector_from_list(lst): Converts a Python list of floats to ql.DoubleVector.
size_vector_from_list(lst): Converts a Python list of integers to ql.SizeVector.
matrix_from_list_of_lists(lol): Converts a Python list of lists to ql.Matrix.
These helpers make the main test code cleaner by encapsulating the conversion from Python native types to QuantLib vector/matrix types.
The assertions use self.assertAlmostEqual with the delta parameter to match the C++ test's tolerance logic directly.
The QL_DEPRECATED_DISABLE_WARNING and QL_DEPRECATED_ENABLE_WARNING macros are C++ preprocessor directives related to managing deprecation warnings during compilation and don't have a direct equivalent or need in the Python test script runtime.